In [ ]:
import sqlite3
import pandas as pd

MOUNTPOINT = '/content/drive'

from google.colab import drive
drive.mount(MOUNTPOINT)

if 'conn' in globals() and isinstance(conn, sqlite3.Connection):
  conn.close()

db_path = f"{MOUNTPOINT}/My Drive/fraud.db"
conn = sqlite3.connect(db_path)

Mounted at /content/drive


In [ ]:
queryStr = """
SELECT name
FROM sqlite_master
WHERE type='table'
;
"""

tables = pd.read_sql_query(queryStr, conn)
tables

,name
0,fraud_scores


In [ ]:
schema = pd.read_sql("PRAGMA table_info(fraud_scores);", conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,TransactionDT,BIGINT,0,None,0
1,1,TransactionAmt,FLOAT,0,None,0
2,2,ProductCD,BIGINT,0,None,0
3,3,card1,BIGINT,0,None,0
4,4,card2,FLOAT,0,None,0
...,...,...,...,...,...,...
419,419,DeviceInfo,BIGINT,0,None,0
420,420,TransactionID,BIGINT,0,None,0
421,421,actual,BIGINT,0,None,0
422,422,predicted,BIGINT,0,None,0


In [ ]:
create_high_risk = """
CREATE VIEW IF NOT EXISTS high_risk_users AS
SELECT
  card1 AS card_id,
  AVG(fraud_probability) AS avg_fraud_score,
  COUNT(*) AS txn_count
FROM fraud_scores
GROUP BY card1
ORDER BY avg_fraud_score DESC
LIMIT 150
;
"""
conn.executescript(create_high_risk)
conn.commit()


### Creates view called high risk users, renames card1 to card_id, calculates average fraud probability, counts number of transations,then group by card id, sorts by risk, and shows top 50

In [ ]:
create_daily_fraud = """
CREATE VIEW IF NOT EXISTS daily_fraud_stats AS
SELECT
    DATE(TransactionDT, 'unixepoch') AS txn_date,
    COUNT(*) AS total_txns,
    SUM(actual) AS fraud_txns,
    ROUND(SUM(actual) * 1.0 / COUNT(*), 4) AS fraud_rate
FROM fraud_scores
GROUP BY txn_date
ORDER BY txn_date;
"""
conn.executescript(create_daily_fraud)
conn.commit()

### Converts to date, counts transactions, counts many were frauds, fraud percentage, and groups by chronological order

In [ ]:
create_fraud_by_product = """
CREATE VIEW IF NOT EXISTS fraud_by_product AS
SELECT
    ProductCD,
    card4,
    DeviceInfo AS browser,
    COUNT(*) AS total_txns,
    SUM(actual) AS fraud_txns,
    ROUND(SUM(actual) * 1.0 / COUNT(*), 4) AS fraud_rate
FROM fraud_scores
GROUP BY ProductCD, card4, browser
ORDER BY fraud_rate DESC;
"""
conn.executescript(create_fraud_by_product)
conn.commit()

### Groups product code, card type, and device info
### Counts transations, counts frauds, and computes fraud rate